In [67]:
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (10,6)
from collections import defaultdict, deque

import math

In [ ]:
import glob

# df = pd.read_csv('../data/raw/euro-2024-2025/B1.csv')
csv_2024 = glob.glob('../data/raw/euro-2024-2025/*.csv')
csv_2023 = glob.glob('../data/raw/euro-2023-2024/*.csv')
csv_2022 = glob.glob('../data/raw/euro-2022-2023/*.csv')
csv_2021 = glob.glob('../data/raw/euro-2021-2022/*.csv')
csv_2020 = glob.glob('../data/raw/euro-2020-2021/*.csv')


# Div = League Division
# Date = Match Date (dd/mm/yy)
# Time = Time of match kick off
# HomeTeam = Home Team
# AwayTeam = Away Team
# FTHG and HG = Full Time Home Team Goals
# FTAG and AG = Full Time Away Team Goals
# FTR and Res = Full Time Result (H=Home Win, D=Draw, A=Away Win)
# HTHG = Half Time Home Team Goals
# HTAG = Half Time Away Team Goals
# HTR = Half Time Result (H=Home Win, D=Draw, A=Away Win)

# Match Statistics (where available)
# Attendance = Crowd Attendance
# Referee = Match Referee
# HS = Home Team Shots
# AS = Away Team Shots
# HST = Home Team Shots on Target
# AST = Away Team Shots on Target
# HHW = Home Team Hit Woodwork
# AHW = Away Team Hit Woodwork
# HC = Home Team Corners
# AC = Away Team Corners
# HF = Home Team Fouls Committed
# AF = Away Team Fouls Committed
# HFKC = Home Team Free Kicks Conceded
# AFKC = Away Team Free Kicks Conceded
# HO = Home Team Offsides
# AO = Away Team Offsides
# HY = Home Team Yellow Cards
# AY = Away Team Yellow Cards
# HR = Home Team Red Cards
# AR = Away Team Red Cards
# HBP = Home Team Bookings Points (10 = yellow, 25 = red)
# ABP = Away Team Bookings Points (10 = yellow, 25 = red)


In [ ]:

def load_csv_files(file_list, index):
    dataframes = pd.DataFrame()
    for file in file_list:
        try:
            non_ascii_pattern = r'[^\x00-\x7F]+'
            df_temp = pd.read_csv(file, encoding='latin1')
            for col in df_temp.select_dtypes(include='object').columns:
                df_temp[col] = df_temp[col].str.replace(non_ascii_pattern, '', regex=True)
            dataframes = pd.concat([dataframes, df_temp], ignore_index=True)
        except Exception as e:
            print(f"Error reading at index {index} for file {file}: {e}")
    return dataframes


In [ ]:
df1=df2=df3=df4=df5=pd.DataFrame()

for i, csv in enumerate([csv_2024, csv_2023, csv_2022, csv_2021, csv_2020], start=1):
    vars()[f'df{i}'] = load_csv_files(csv, i)

In [ ]:
for i in range(1, 6):
    print(f"DataFrame df{i} shape: {vars()[f'df{i}'].shape}")  # Print the shape of each DataFrame

df1['Div'] = df1['ï»¿Div']
df1.drop('ï»¿Div', axis=1, inplace=True)

In [ ]:
df1.columns.to_list()  

In [ ]:
# Collecting all stats that's relevant to the result

# Div = League Division
# Date = Match Date (dd/mm/yy)
# Time = Time of match kick off
# HomeTeam = Home Team
# AwayTeam = Away Team
# FTHG and HG = Full Time Home Team Goals
# FTAG and AG = Full Time Away Team Goals
# FTR and Res = Full Time Result (H=Home Win, D=Draw, A=Away Win)
# HTHG = Half Time Home Team Goals
# HTAG = Half Time Away Team Goals
# HTR = Half Time Result (H=Home Win, D=Draw, A=Away Win)
# Referee = Match Referee
# HST = Home Team Shots on Target
# AST = Away Team Shots on Target
# HC = Home Team Corners
# AC = Away Team Corners
# HR = Home Team Red Cards
# AR = Away Team Red Cards

relevant_columns = ['Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR', 'HST', 'AST', 'HC', 'AC', 'HR', 'AR']

ft_stat_df1 = df1[relevant_columns]


In [ ]:
# assigning shots on target to na columns
zero_column_field = ['HR','AR','HTHG','HTAG','HC','AC']

ft_stat_df1['HST'].fillna(np.ceil(ft_stat_df1['FTHG']*1.5),inplace=True)
ft_stat_df1['AST'].fillna(np.ceil(ft_stat_df1['FTAG']*1.5),inplace=True)
ft_stat_df1[zero_column_field] = ft_stat_df1[zero_column_field].fillna(0)
ft_stat_df1['Date'] = pd.to_datetime(ft_stat_df1['Date'], dayfirst=True)

# dropping row where htr is na
ft_stat_df1 = ft_stat_df1[~ft_stat_df1['HTR'].isna()]

In [ ]:
ft_stat_df1.head(10)

form_window = 5

playing_stats = ['points', 'pos',  'gf', 'ga', 'gp', 'form', 'perf']



In [ ]:

# In a given season, taking e0 as example, each team plays 38 matches, which makes total of 380 matches.

# first, let's sort the  data by the date
pro_data = ft_stat_df1.head(20).copy()
# pro_data = ft_stat_df1.sort_values([ 'Date','Div']).reset_index(drop=False)

# initialize columns
out_cols = ['home_pos', 'away_pos', 'home_pt', 'away_pt', 'home_gf', 'home_ga', 'away_gf', 'away_ga', 'home_form', 'away_form', 'home_gp', 'away_gp'  ]
for c in out_cols:
        pro_data[c] = pd.NA

# # last ten matches for each teams
form_window = 5

# i want -
# home_pos = 2 - home position in league table
# home_pt = 25 - home league points
# home_gp = 10 - home total games played in the league
# home_gf = 22 - home scored goals
# home_ga = 8 - home goal against/conceded goals
# home_form = 'wwlwdwwdw' - home form overall in the league
# home_hp = "wwdwd" - home home performance (their performance when playing on their home soil)
# leage_p = 10 - league progress (percentage completion of the league)


# group datasets by div and sort in descending order (first (will be old) games comes to top)
pro_data_sorted = pro_data.sort_values(by=['Div', 'Date'], ascending=[True, True])

processed_df = pd.DataFrame(index=pro_data_sorted.index)
for div, group in pro_data_sorted.groupby(['Div'], sort=False):
        team_stats_after_match = team_stats_before_match = defaultdict(lambda: {'points': 0, 'pos': 0,  'gf': 0, 'ga': 0, 'gp': 0, 'form': deque(maxlen=form_window), 'perf': deque(maxlen=form_window)})
        counter = 0
        for date, day_group in group.groupby(['Date'], sort=False): #match days
                #calculate points for each teams before matches
                for row in day_group.itertuples(index=True): #matches on a given day
                        index = row.Index
                        home = row.HomeTeam
                        away = row.AwayTeam
                        hg = row.FTHG
                        ag = row.FTAG
                        
                        # team_stats_before_match = team_stats_after_match
                        
                        if hg > ag:
                                # processed_df.loc[index, 'home_pt'] += team_stats[home]['points']
                                # team_stats_before_match[home]['points'] += team_stats_after_match[home]['points']
                                team_stats_after_match[home]['points'] += 3
                        elif ag > hg:
                                # processed_df.loc[index, 'away_pt'] += team_stats[away]['points']
                                # team_stats_before_match[away]['points'] += team_stats_after_match[away]['points']
                                team_stats_after_match[away]['points'] += 3
                        else:
                                # # processed_df.loc[index, 'home_pt'] += team_stats[home]['points']
                                # team_stats_after_match[home]['points'] += team_stats_after_match[home]['points']
                                # team_stats_after_match[away]['points'] += team_stats_after_match[away]['points']
                                team_stats_after_match[home]['points'] += 1
                                team_stats_after_match[away]['points'] += 1
                      
                        # Goals for/against
                        team_stats_after_match[home]['gf'] += hg
                        team_stats_after_match[home]['ga'] += ag
                        team_stats_after_match[away]['gf'] += ag
                        team_stats_after_match[away]['ga'] += hg
                        
                        # Games played
                        team_stats_after_match[home]['gp'] += 1
                        team_stats_after_match[away]['gp'] += 1
                        
                        # print(f'match day {date}')   
                        # print(f'{home} vs {away} forebet ')
                        # print(f'{home} : gf - {team_stats[home]['gf']}, ga - {team_stats[home]['ga']}')
                        # print(f'{home} : gf - {team_stats[away]['gf']}, ga - {team_stats[home]['ga']}')
                        # print(' ----- ----  - - - - - - ')
                        counter += 1

                #calculate positions for each teams after each matchday
                sorted_teams = sorted(
                        team_stats_after_match.items(),
                        key=lambda x: (x[1]['points'], x[1]['gf'] - x[1]['ga'], x[1]['gf']),
                        reverse=True
                        )
                # rank_mapping = {team: i + 1 for i, (team, _) in enumerate(sorted_teams)}

                # print(f'before: {team_stats_before_match}')
                # print(f'after: {team_stats_after_match}')
                # print('0----------------0')
                print(counter)
               
# processed_df.head(20)
                        
                



# print(f'first: {pro_data_sorted.loc[19, ['Date', 'HomeTeam', 'AwayTeam', 'FTHG']].to_list()}')
# print(f'first: {pro_data_sorted.loc[0, ['Date', 'HomeTeam', 'AwayTeam', 'FTHG']].to_list()}')
# pro_data_sorted.head(20)[['HomeTeam', 'Date']]


In [ ]:
# team_stats
# ft_stat_df1.columns
# pro_data = ft_stat_df1.sort_values(['Div', 'Date']).reset_index(drop=False)
ft_stat_df1.head(15)

In [ ]:
def get_form(team, dataset, n: int):
    team_matches = dataset[(dataset['HomeTeam'] == team) | (dataset['AwayTeam'] == team)]
    team_matches = team_matches.sort_values(by='Date', ascending=False).head(n)
    
    results = []
    for _, match in team_matches.iterrows():
        if match['HomeTeam'] == team:
            if match['FTR'] == 'H':
                results.append('W')
            elif match['FTR'] == 'D':
                results.append('D')
            else:
                results.append('L')
        else:
            if match['FTR'] == 'A':
                results.append('W')
            elif match['FTR'] == 'D':
                results.append('D')
            else:
                results.append('L')
    
    return results


def get_goals_scored_concede_in_n_match(team, n: int, dataset):
    team_matches = dataset[(dataset['HomeTeam'] == team) | (dataset['AwayTeam'] == team)]
    team_matches = team_matches.sort_values(by='Date', ascending=False).head(n)
    
    goals_scored = 0
    goals_conceded = 0
    
    for _, match in team_matches.iterrows():
        if match['HomeTeam'] == team:
            goals_scored += match['FTHG']
            goals_conceded += match['FTAG']
        else:
            goals_scored += match['FTAG']
            goals_conceded += match['FTHG']
    
    return goals_scored, goals_conceded

def home_advantage(team, dataset):
    home_matches = dataset[dataset['HomeTeam'] == team]
    away_matches = dataset[dataset['AwayTeam'] == team]
    
    home_wins = home_matches[home_matches['FTR'] == 'H'].shape[0]
    away_wins = away_matches[away_matches['FTR'] == 'A'].shape[0]
    
    total_home_matches = home_matches.shape[0]
    total_away_matches = away_matches.shape[0]
    
    home_win_rate = home_wins / total_home_matches if total_home_matches > 0 else 0
    away_win_rate = away_wins / total_away_matches if total_away_matches > 0 else 0
    
    return home_win_rate, away_win_rate 

def leageue_position():
    pass

def team_strength():
    pass



# for each match, the following features are to be created:


In [ ]:

# In a given season, taking e0 as example, each team plays 38 matches, which makes total of 380 matches.

# first, let's sort the  data by the date
pro_data = ft_stat_df1.sort_values(['Div', 'Date']).reset_index(drop=False)

# initialize columns
out_cols = ['home_pos', 'away_pos', 'home_pt', 'away_pt', 'home_gf', 'home_ga', 'away_gf', 'away_ga', 'home_form', 'away_form',  ]
for c in out_cols:
    pro_data[c] = pd.NA

# last ten matches for each teams
form_window = 10

# now the fun part yeaaaa
for _, group in pro_data.groupby(['Div'], sort=False):
    group = group.sort_values('Date')
    team_stats_after_match = defaultdict(lambda: {'points':0, 'gf':0, 'ga':0, 'gp':0})
    recent = defaultdict(lambda: deque(maxlen=form_window))

    for index, row in pro_data.iterrows():
        orig_idx = row['index']           # original df index where we'll write results
        home,away = row['HomeTeam'], row['AwayTeam']
        hg, ag = row['FTHG'], row['FTAG']
        
        # let's ensure each teams exist in team_stats
        _ = team_stats_after_match[home]; _ = team_stats_after_match[away]


         # computing each teams league position based on 
        #     1 - points
        #     2 - goal difference (gf-ga)
        #     3 - gf if there's a tie after above two conditions...

        #before this match, let's build the league(division) table first
        # my god, this is prolly most confusing bit i ever wrote in this notebook yet. basically does what the above comment says 🤭
        table = sorted(team_stats_after_match.items(), key=lambda x: (x[1]['points'], x[1]['gf']-x[1]['ga'], x[1]['gf']), reverse=True)
        
        
        pos = {team:i+1 for i,(team,_) in enumerate(table)}

        default_pos = len(table) + 1
        
        pro_data.at[orig_idx, 'home_pos'] = pos.get(home, default_pos)
        pro_data.at[orig_idx, 'away_pos'] = pos.get(away, default_pos)
        pro_data.at[orig_idx, 'home_pt'] = team_stats_after_match[home]['points']
        pro_data.at[orig_idx, 'away_pt'] = team_stats_after_match[away]['points']
        pro_data.at[orig_idx, 'home_gf'] = team_stats_after_match[home]['gf']
        pro_data.at[orig_idx, 'home_ga'] = team_stats_after_match[home]['ga']
        pro_data.at[orig_idx, 'away_gf'] = team_stats_after_match[away]['gf']
        pro_data.at[orig_idx, 'away_ga'] = team_stats_after_match[away]['ga']
        
        # form: average points over last form window matches (or NA if none)
        pro_data.at[orig_idx, 'home_form'] = (sum(recent[home]) / len(recent[home])) if len(recent[home])>0 else pd.NA
        pro_data.at[orig_idx, 'away_form'] = (sum(recent[away]) / len(recent[away])) if len(recent[away])>0 else pd.NA
        

        # updating goals for each teams
            # home team
        team_stats_after_match[home]['gf'] += hg
        team_stats_after_match[home]['ga'] += ag
            # away team
        team_stats_after_match[away]['gf'] += ag
        team_stats_after_match[away]['ga'] += hg


        # updating points for each teams
        if hg > ag: #home won, cos home scored more goals than away... simple yea?
            team_stats_after_match[home]['points'] +=3
            recent[home].append(3); recent[away].append(0)
        elif ag > hg: # away won
            team_stats_after_match[away]['points'] +=3
            recent[home].append(0); recent[away].append(3)
        else: #match ends in draw
            team_stats_after_match[away]['points'] += 1
            team_stats_after_match[home]['points'] += 1
            recent[home].append(1); recent[away].append(1)
            
            
        # update number of matches played for each teams
        team_stats_after_match[home]['gp'] += 1
        team_stats_after_match[away]['gp'] += 1

        # computing each teams league position based on 
        #     1 - points
        #     2 - goal difference (gf-ga)
        #     3 - gf if there's a tie after above two conditions...

        # my god, this is prolly most confusing bit i ever wrote in this notebook yet. basically does what the above comment says 🤭
        # table = sorted(team_stats.items(), key=lambda x: (x[1]['points'], x[1]['gf']-x[1]['ga'], x[1]['gf']), reverse=True)
        # pos = {team:i+1 for i,(team,_) in enumerate(table)}
        # pro_data.at[index, 'home_pos'] = pos[home]
        # pro_data.at[index, 'away_pos'] = pos[away]


pro_data.head(10)
